# Qualitätsprüfung der Transkriptom-Daten
## Definition 'Ausreißer'
Um später unser Modell mit korrekten Datensätzen zu trainieren, werden hier zunächst die full_raw_counts-Tabellen aller Datensätze um 'Ausreißer' bereinigt.
Unter Ausreißer werden gefasst: 
1.	Fehlende Werte: leere Tabellenzellen
2.	GeneIDs, die zu allen Phasen gleich stark exprimiert werden und somit keine Hilfe für die Klassifikation in früh-mittel-spät bieten
3.	(Falls mehrer Runs in einem Datensatz vorliegen: Auffällige Abweichungen der Expression eines Gens in Relation zu den anderen Runs: Mittelwerte zu jeder GeneID pro Probe und jeweils miteinander vergleichen um „schlechte“ (fehlerhafte Proben, Messfehler, Pippetierfehler) Runs zu filtern) #TODO
4. Host-Sätze, da das Programm auf die Klassifikation der Phagengene abzielt
5. Gene, die zu irgendeinem Zeitpunkt einen Expressionscount von 0 aufweisen, da keine sinnvolle Verwertung möglich ist  

## Detektion
1. Fehlende Werte: pandas NaN
2. GeneIDs, die zu allen Phasen ähnlich (z.B. < 5% Schwankung über alle Phasen) stark exprimiert werden: pandas und statistische Methoden
3. (Abweichungen zwischen den Runs: pandas und statistische Methoden) #TODO
4. Host-Sätze markieren: pandas
5. Gene, die zu irgendeinem Zeitpunkt einen Expressions-count von 0 aufweisen: pandas

## Kenntlichmachung
Die markierten count-Tabellen werden im jeweiligen Unterordner data/**/*_full_raw_counts_marked.tsv abgelegt. 
Ausreißer in Tabelle markiert durch 1 in Spalte 'Outlier', genauer:
1. Markierung der leeren Zellen mit -1, 1 in Spalte 'Rows with missing values'
2. Markierung von Genen, deren counts sich über alle Phasen kaum unterscheiden mit 1 in Spalte 'negligible changes'
3. # TODO abweichende Werte zwischen den Replikaten
4. Markierung von Host-Genen durch eine 1 in Spalte 'Outlier'
5. Markierung von Genen, die zu irgendeinem Zeitpunkt einen Expressions-count von 0 aufweisen mit 1 in 'Expression of 0'
Außerdem wird unter "data/marked/overview_marks.txt jeder count-Datensatz unter Angabe der Zahl der Ausreißer aufgeführt.
------------------------------------------------------------------------------------

In [3]:
import pandas as pd
import numpy as np
import glob
import os
import glob

datasets = [
    "Yang",
    "Sprenger_VC_WT_VP882_delta_cpdS",
    "Lood",
    "Guegler_T7_plusToxIN",
    "Guegler_T4_minusToxIN",
    "Finstrlova_Newman",
    "Brandao_MCCM"
]

tsv_dateien = []
for ds in datasets:
    pattern = f"./data/**/{ds}_full_raw_counts.tsv"
    tsv_dateien.extend(glob.glob(pattern, recursive=True))
threshold = 0.05 # Schwellenwert, wenn Varianz < Schwellenwert wird das Gen als vernachlässigbar eingestuft

def detect_empty_cells(df):
    ''' 
    Markiert leere Zellen mit -1,
    gibt den DataFrame zurück, die Gesamtzahl der leeren Zellen
    und ein Array (Liste), das für jede Zeile speichert, ob dort mindestens eine leere Zelle war (1) oder nicht (0)
    '''
    missing_values = df.isna().sum().sum()
    # Array: 1 falls mindestens eine NaN in der Zeile, sonst 0
    rows_with_missing = df.isna().any(axis=1).astype(int).tolist()
    df = df.fillna(-1)
    return df, missing_values, rows_with_missing

def detect_relative_changes(df, count_cols, threshold):
    '''
    berechnet, wie stark die Daten innerhalb einer Zeile (also eines Gens über die gesamte Dauer) variiert
    '''
    relative_changes = [] # Wert für jede Zeile, wie stark Expression des Gens variiert über die Zeit
    negligible_changes = [] # für jede Zeile 1 wenn Wert weniger als der angegebene Schwellenwert variiert, sonst 0
    for _, row in df.iterrows():
        numbers = []
        for val in row[count_cols]:
            try:
                numbers.append(float(val))
            except ValueError:
                continue
        if numbers and np.mean(numbers) != 0:
            mean = np.mean(numbers)
            std = np.std(numbers)
            relation = std / mean
            relative_changes.append(relation)
            negligible_changes.append(1 if relation < threshold else 0) # wenn Varianz kleiner als Schwellenwert wird die Zeile als vernachlässigbar bewertet
        else:
            relative_changes.append(0)
            negligible_changes.append(0)
    return relative_changes, negligible_changes

def detect_rows_with_zeroes(df, count_cols):
    '''
    Prüfe für jede Zeile, ob mindestens ein Wert in den Count-Spalten 0 ist
    '''
    ausreisser = (df[count_cols] == 0).any(axis=1).astype(int)
    return ausreisser

def collect_outliers(df):
    '''
    ergänzt Spalte 'Outlier' und markiert dort alle Zeilen/Gene mit 1, in denen ein Ausreißer entdeckt wurde
    '''
    df["Outlier"] = (
    (df["negligible changes"] == 1) |
    (df["Expression of 0"] == 1) |
    (df["Rows with missing values"] == 1) |
    (df["Entity"] == "host") # Outlier auch, wenn in Spalte 'entity' 'host' steht
    ).astype(int)
    return df

def save_table(df, original_path, suffix="_marked.tsv"):
    '''
    speichert DataFrame im Ordner data/marked als _marked.tsv
    '''
    outdir = "data/marked"
    os.makedirs(outdir, exist_ok=True)
    basename = os.path.basename(original_path)
    outname = os.path.join(outdir, basename.replace('.tsv', suffix))
    df.to_csv(outname, sep='\t', index=False, encoding='utf-8')
    return outname

quality_information = [] # sammelt für jeden Datensatz das Vorkommen von Ausreißern

for datei in tsv_dateien:
    count_table = df = pd.read_csv(datei, sep='\t')
    count_table, missing_values, rows_with_missing = detect_empty_cells(count_table)
    count_cols = count_table.columns[1:-2] # Counts liegen von Spalte 1 bis -2. Annahme: Die erste Spalte gibt GeneID an, die letzten beiden sind Textspalten (in allen raw_counts Tabellen). 
    relative_changes, negligible_changes = detect_relative_changes(count_table, count_cols, threshold)
    rows_with_zeroes = detect_rows_with_zeroes(count_table, count_cols)
    count_table["relative changes"] = relative_changes # hängt als Spalte an
    count_table["negligible changes"] = negligible_changes # hängt als Spalte an
    count_table["Expression of 0"] = rows_with_zeroes
    count_table["Rows with missing values"] = rows_with_missing
    negligible_genes = sum(negligible_changes)
    genes_with_zero_expression = sum(rows_with_zeroes)
    quality_information.append(
        f"{datei}: missing values: {missing_values}, negligible genes: {negligible_genes}, genes with expression of 0: {genes_with_zero_expression}"
    )
    count_table = collect_outliers(count_table)
    save_table(count_table, datei)

with open("data/marked/overview_marks.txt", "w") as f:
    for line in quality_information:
        f.write(line + "\n")










ModuleNotFoundError: No module named 'pandas'